In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

Start with the Nashville zipcode outlines and the Davidson County Boundary, then find the centroid of the county to center the map with.

In [2]:
all_crashes = pd.read_csv('../data/clean/all_crashes_maps.csv')
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
county

,area_sq_mi,objectid,shape_area,shape_len,geometry
0,525,4,14641027913.6,686109.334953,"MULTIPOLYGON (((-86.52704 36.13598, -86.52914 ..."
1,0,3,675823.52,3786.70930403,"MULTIPOLYGON (((-86.52315 36.13876, -86.52288 ..."


In [3]:
county = gpd.read_file('../data/clean/Davidson County Border (GIS).geojson')
# Use ID 0 for centroid
center = county.geometry.centroid[0]
map_center = [center.y, center.x]
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
crash_data_geo = all_crashes[['accident_number', 'lat', 'long']]
crashes_geo = gpd.GeoDataFrame(crash_data_geo, crs= zipcodes.crs, geometry=gpd.points_from_xy(crash_data_geo.long, crash_data_geo.lat))

C:\Users\chabi\AppData\Local\Temp\ipykernel_9316\3216260469.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = county.geometry.centroid[0]


Nothing jumping out as a hot spot from te hheatmap. Try using a marker cluster.

In [4]:
all_crashes_cluster = folium.Map(location = map_center, zoom_start = 10)

marker_cluster = MarkerCluster().add_to(all_crashes_cluster)

folium.GeoJson(zipcodes).add_to(all_crashes_cluster)

for row_index, row_values in crashes_geo.iterrows():
    loc = [row_values['lat'], row_values['long']]
    icon=folium.Icon(color="orange",icon="exclamation-triangle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
    icon = icon) 
    
    marker.add_to(marker_cluster)

all_crashes_cluster.save('../maps/all_crashes_cluster.html')